In [ ]:
from argparse import ArgumentParser
from itertools import combinations as comb
from os import listdir, makedirs
from os.path import exists, isfile, join

import cv2 as cv
import numpy as np
from detector import Detector
from logger import Logger, _timestamp
from marker import Marker
from natsort import natsorted
from PIL import Image
from tqdm import tqdm
from utils import load, save

**Defining Settings**

-   `DICT_TYPE`: ArUCo marker dictionary
-   `MARKER_SIZE`: Marker size (can be in any unit)
-   `VALID_MARKER_IDS`: Detection will only match the IDs provided below, leave the list empty if you want possible markers to be detected, highly advised to be used if `USE_ADVANCED_THRESHOLDING` is enabled
-   `ROOT_DIRECTORY`: Root directory of the images, see below for file structure
-   `OUTPUT_DIRECTORY`: Output directory for the results, log file, and generated images
-   `GENERATE_RESULTS`: Boolean flag for generating result images
-   `GENERATE_OVERLAYS`: Boolean flag for generating an overlaid result of all the result images of the subdirectory
-   `USE_ADVANCED_THRESHOLDING`: Boolean flag for enabling advanced thresholding. Sweeps for a multitude of thresholding settings, aids in detecting markers that are less visible.
-   `DETECTION_ONLY`: Boolean flag for disabling comparison and calculations
-   `VERBOSE_OUTPUT`: Boolean flag for enabling command line output

**File Structure**
The images should be structured in the following directory format.
Each subdirectory must have two or more images.

```
/path/to/root/folder
    |--> /sub_directory_1
        |--> repeat1.jpg
        |--> repeat2.jpg
    |--> /sub_directory_2
        |--> repeat1.jpg
        |--> repeat2.jpg
    |--> /sub_directory_3
        |--> repeat1.jpg
        |--> repeat2.jpg
        |--> repeat3.jpg
    ...
```


In [ ]:
DICT_TYPE = "DICT_4X4_100"
MARKER_SIZE = 3
VALID_MARKER_IDS = [0, 1, 2, 3, 4, 5, 6, 7]

ROOT_DIRECTORY = "data/"
OUTPUT_DIRECTORY = "out/"

GENERATE_RESULTS = True
GENERATE_OVERLAYS = True

USE_ADVANCED_THRESHOLDING = False
DETECTION_ONLY = False

VERBOSE_OUTPUT = True

logger = Logger(OUTPUT_DIRECTORY)
detector = Detector(DICT_TYPE, USE_ADVANCED_THRESHOLDING, VALID_MARKER_IDS, logger)

**Initialisation Step:**

-   Verifying that each subdirectory of the root has two or more images
-   Create leaf directories in the output folder


In [ ]:
logger.plain("=== INITIALISATION: Verifying Subdirectories ===")

markers = {}
imgs = []

subDirs = natsorted(listdir(ROOT_DIRECTORY))
iterator = subDirs if VERBOSE_OUTPUT else tqdm(subDirs)

for subDirName in iterator:
    logger.action(f"Checking folder: {subDirName}")
    folder = join(ROOT_DIRECTORY, folder)
    fList = natsorted(listdir(folder))

    if len(fList) < 2:
        logger.warn(f"Skipping directory - directory has less than 2 images")

    markers[subDirName] = {}
    for fileName in natsorted(listdir(folder)):
        file = join(folder, fileName)
        markers[subDirName][fileName] = {}
        imgs.append((subDirName, fileName))

    if GENERATE_RESULTS:
        res = join(OUTPUT_DIRECTORY, "images", subDirName)
        makedirs(res)

        if GENERATE_OVERLAYS:
            makedirs(join(res, "overlay"))

In [ ]:
logger.plain("=== DETECTION ===")

iterator = imgs if VERBOSE_OUTPUT else tqdm(imgs)
for subDirName, fileName in iterator:
    ids, markers = detector.detectMarkersFromFile(join(ROOT_DIRECTORY, subDirName, fileName))
    markers[subDirName][fileName] = dict(zip(ids, markers))